In [ ]:
# Imports e arquivos
import pandas as pd
import matplotlib.pyplot as plt

path = r"C:\Users\Samsung\Desktop\ZZZ\DESPESAS_CEAPS-2008-2022.csv"
file = pd.read_csv(path, sep=';',low_memory=False)

In [ ]:
# Transformando a coluna "VALOR_REEMBOLSADO" de string para float
file['VALOR_REEMBOLSADO'] = file['VALOR_REEMBOLSADO'].astype(str)
file['VALOR_REEMBOLSADO'] = file['VALOR_REEMBOLSADO'].str.replace(',','.')
file['VALOR_REEMBOLSADO'] = file['VALOR_REEMBOLSADO'].str.replace('\r\n','')
file['VALOR_REEMBOLSADO'] = file['VALOR_REEMBOLSADO'].astype(float)
    
# Removendo colunas desnecessárias para a atividade
df = file.drop(['CNPJ_CPF','FORNECEDOR','DOCUMENTO','DATA','DETALHAMENTO','COD_DOCUMENTO'],axis=1)
display(df)

In [ ]:
# Função para remover itens repetidos das listas criadas a partir das colunas do DataFrame, sem perder a ordem original
def removeRepetidos(lista):
    visto = set()
    adicionar_visto = visto.add
    return [x for x in lista if not (x in visto or adicionar_visto(x))]

In [ ]:
# Criando listas a partir das colunas do DataFrame
lista_ano = df['ANO'].tolist()
lista_mes = df['MES'].tolist()
lista_senadores = df['SENADOR'].tolist()
lista_tipo_despesa = df['TIPO_DESPESA'].tolist()
lista_gasto_medio_mes = []
lista_gasto_medio_despesa = []

# Descobrindo os gastos médios por mês
for mes in removeRepetidos(lista_mes):
    df_mes = df[df['MES']==mes]
    gasto_medio_mes = (df_mes['VALOR_REEMBOLSADO'].sum())/len(df_mes)
    lista_gasto_medio_mes.append(gasto_medio_mes)

# Descobrindo os gastos médios por tipo de despesa
for tipo_despesa in removeRepetidos(lista_tipo_despesa):
    df_despesa = df_mes[df_mes['TIPO_DESPESA']==tipo_despesa]
    gasto_medio_despesa = (df_despesa['VALOR_REEMBOLSADO'].sum())/len(df_despesa)
    lista_gasto_medio_despesa.append(gasto_medio_despesa)

In [ ]:
# Gráfico de gastos médios por mês
fig = plt.figure(figsize=(15,8))
eixo = fig.add_axes([0, 0, 1, 1])
eixo.plot(removeRepetidos(lista_mes),lista_gasto_medio_mes, color="blue")
eixo.set_title(("Gasto médio por mês"), fontsize=25)
eixo.set_ylabel("Gasto em reais", fontsize=15)
eixo.set_xlabel("Mês", fontsize=15)
eixo.legend(["Gasto"], loc='lower right', fontsize=10)

In [ ]:
# Gráfico de gastos médios por tipo de despesa
fig = plt.figure(figsize=(15,8))
eixo = fig.add_axes([0, 0, 1, 1])
eixo.plot(lista_gasto_medio_despesa,removeRepetidos(lista_tipo_despesa), color='red')
eixo.set_title(("Gasto médio por tipo de despesa"), fontsize=25)
eixo.set_ylabel("Tipo de despesa", fontsize=20)
eixo.set_xlabel("Gasto em reais", fontsize=20)
eixo.legend(["Gasto"], loc='lower right', fontsize=15)

In [ ]:
# Verificação 
media_total_gastos = df['VALOR_REEMBOLSADO'].sum()/len(df)
print(media_total_gastos,"é a média total de gastos de todos os senadores, de 2008 à 2022")
print("Os gastos desses tipos estão acima da média:")
for j,k in zip(lista_gasto_medio_despesa,removeRepetidos(lista_tipo_despesa)):
    if j > media_total_gastos:
        print(k)

In [ ]:
# (tentativa 3)
for ano in removeRepetidos(lista_ano):
    df_ano = df[df['ANO']==ano]
    lista_senadores_ano = df_ano['SENADOR'].tolist()
    for senador in removeRepetidos(lista_senadores_ano):
        lista_medias_despesas = []
        soma_medias = 0
        variancia = 0
        df_senadores_ano = df_ano[df_ano['SENADOR']==senador]
        lista_senadores_ano_despesa = df_senadores_ano['TIPO_DESPESA'].tolist()
        for tipo_despesa in removeRepetidos(lista_senadores_ano_despesa):
            df_senadores_ano_despesa = df_senadores_ano[df_senadores_ano['TIPO_DESPESA']==tipo_despesa]
            lista_gasto_senadores_ano_despesa = df_senadores_ano_despesa['VALOR_REEMBOLSADO'].tolist()
            gasto_medio_senadores_ano_despesa = df_senadores_ano_despesa['VALOR_REEMBOLSADO'].sum()/len(df_senadores_ano_despesa)
            lista_medias_despesas.append(gasto_medio_senadores_ano_despesa)
            soma_medias += gasto_medio_senadores_ano_despesa
        media_anual_senador = soma_medias/len(lista_medias_despesas)
        for x in lista_medias_despesas:
            variancia += ((x-media_anual_senador)**2)/len(lista_medias_despesas)
        desvio_padrao = variancia**(1/2)

        fig = plt.figure(figsize=(15,8))
        eixo = fig.add_axes([0, 0, 1, 1])
        eixo.plot(removeRepetidos(lista_senadores_ano_despesa), lista_medias_despesas, color="black")
        eixo.set_title((r"Média dos gastos do(a) senador(a)",senador,"por tipo de despesa no ano de",ano,"e com Desvio Padrão de",desvio_padrao), fontsize=25)
        eixo.set_ylabel("Média de gastos", fontsize=15)
        eixo.set_xlabel("Tipo de despesa", fontsize=15)
        eixo.legend(["Gasto"], loc="lower right", fontsize=10)
